
1. Model name: **A Simple Linear Graph Filtering**
2. Domain: Collaborative filtering; Recommender system
2. Writer: Jin-Duk Park (https://jindeok.github.io/jdpark/)

In [5]:
import torch
import scipy.sparse as sp
from utils_torch_GF import *
import os

In [6]:
## Configuration and Dataloader

# Set your device
device = 'cuda:0' # your cuda device (e.g., 'cuda:0' or 'cpu')

# Specify & load your dataset.
current_directory = os.getcwd()
dataset ="ml-1m" # "ml-1m", "gowalla", "yelp", "amazon"
path_tr = f"{current_directory}/dataset/{dataset}_train.npz"
path_ts = f"{current_directory}/dataset/{dataset}_test.npz"
R_tr = csr2torch(sp.load_npz(path_tr)).to(device)
R_ts = csr2torch(sp.load_npz(path_ts)).to(device)

# Check statistics of the dataset
n_users = R_tr.shape[0]
n_items = R_tr.shape[1]
n_inters = torch.nonzero(R_tr._values()).cpu().size(0) +torch.nonzero(R_ts[0]._values()).cpu().size(0)

print(f"number of users: {n_users}")
print(f"number of items: {n_items}")
print(f"number of overall ratings: {n_inters}")

number of users: 5949
number of items: 2810
number of overall ratings: 403282


In [10]:
## Inference

# Graph processing
mceg_norm = normalize_sparse_adjacency_matrix(R_tr.to_dense(), 0.5)
R = R_tr.to_dense()
P = mceg_norm.T @ mceg_norm
P = torch.tensor(P, dtype = torch.float32, device = device)
R = torch.tensor(R, dtype = torch.float32, device = device)

# Linear GF
results = R@P
gt_mat = R_ts.to_dense()

/tmp/ipykernel_421118/3502622133.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  P = torch.tensor(P, dtype = torch.float32, device = device)
/tmp/ipykernel_421118/3502622133.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  R = torch.tensor(R, dtype = torch.float32, device = device)


In [8]:
## Evaluation

# Evaluation only on the unseen items
results = results + (-99999)*R_tr.to_dense() 
gt_mat = gt_mat.cpu().detach().numpy() # Ground truth matrix
results = results.cpu().detach().numpy() # Prediction

# You can specify 'k'
print(f"Recall@10: {recall_at_k(gt_mat, results, k=10):.4f} \n")
print(f"NDCG@10: {ndcg_at_k(gt_mat, results, k=10):.4f} \n")

Recall@10: 0.0827 

NDCG@10: 0.0745 



In [9]:
## Check GPU memory allocation 
print("Allocated GPU Memory: ", torch.cuda.memory_allocated() / (1024 * 1024), "MB")

Allocated GPU Memory:  8.125 MB
